# Symbolic Stencils

Suppose you have to discretize a differential operator like

$$
\frac{\partial^2}{\partial x_0^2} +2 \frac{\partial^2}{\partial x_0\partial x_1} + \frac{\partial^2}{\partial x_1^2}
$$

but you don't want to use the `FinDiff` objects, for example because you want to
transfer the discretization scheme to some other programming language or you
need the discretized operator for some analytic work. This is where symbolic
`Stencil`s come in handy. `Stencil`s allow to discretize differential operators
of the form

$$
\sum_i c_i \prod_{j=1}^N \frac{\partial^{n_j}}{\partial x_j^{n_j}}
$$

based on a given set of offsets. Here, $N$ is the number of dimensions of space,
$c_i$ are real constants and $n_j$ are non-negative integers.

There are two ways to specify such a differential operator for a `Stencil` object: by dict and by symbolics.
Let's start with symbolics.

In [20]:
from findiff.symbolics import DerivativeSymbol as D # stands for the partial derivative operator
from findiff import Stencil
from itertools import product

offsets = list(product([-1, 0, 1], repeat=2))

diff = D(0, 2) + 2 * D(0) * D(1) + D(1, 2)
stencil = Stencil(offsets, diff, spacings=[1, 1], symbolic=True)
stencil

{(-1, -1): 1/2, (-1, 0): 1, (-1, 1): -1/2, (0, -1): 1, (0, 0): -4, (0, 1): 1, (1, -1): -1/2, (1, 0): 1, (1, 1): 1/2}

Alternatively, instead of using `DerivativeSymbol`, we could have defined the same operator as a dict:

In [2]:
diff = {(2, 0): 1, (1, 1): 2, (0, 2): 1}
stencil = Stencil(offsets, diff, spacings=[1, 1], symbolic=True)
stencil

{(-1, -1): 1/2, (-1, 0): 1, (-1, 1): -1/2, (0, -1): 1, (0, 0): -4, (0, 1): 1, (1, -1): -1/2, (1, 0): 1, (1, 1): 1/2}

Here, each key stands for a term in the sum, the entries in the key-tuple denote the $n_j$ and the values are the $c_i$.

The accuracy (actually error order) can be obtained from the `accuracy` property:

In [3]:
stencil.accuracy

2

If your grid has different spacing along the various axes, you can define that with the `spacing` argument,
which takes a list with a length equal to the number of dimensions. As spacings, you can also define symbolic
values, like

In [4]:
stencil = Stencil(offsets, diff, spacings=[r'\Delta x', r'\Delta y'], symbolic=True)
stencil

{(-1, -1): 1/(2*\Delta x*\Delta y), (-1, 0): \Delta x**(-2), (-1, 1): -1/(2*\Delta x*\Delta y), (0, -1): \Delta y**(-2), (0, 0): (-2*\Delta x**2 - 2*\Delta y**2)/(\Delta x**2*\Delta y**2), (0, 1): \Delta y**(-2), (1, -1): -1/(2*\Delta x*\Delta y), (1, 0): \Delta x**(-2), (1, 1): 1/(2*\Delta x*\Delta y)}

You can access single entries of the stencil with the bracket notation:

In [5]:
stencil[-1, -1]

1/(2*\Delta x*\Delta y)

or loop over all values using the `keys()` method:

In [8]:
from IPython.display import display

for key in stencil.keys():
    display(stencil[key])

1/(2*\Delta x*\Delta y)

\Delta x**(-2)

-1/(2*\Delta x*\Delta y)

\Delta y**(-2)

(-2*\Delta x**2 - 2*\Delta y**2)/(\Delta x**2*\Delta y**2)

\Delta y**(-2)

-1/(2*\Delta x*\Delta y)

\Delta x**(-2)

1/(2*\Delta x*\Delta y)

And finally, you can convert the stencil into a Sympy expression for further processing:

In [12]:
expr, symbols = stencil.as_expression()
expr

u[i_0, i_1 + 1]/\Delta y**2 + u[i_0, i_1 - 1]/\Delta y**2 + u[i_0 + 1, i_1 + 1]/(2*\Delta x*\Delta y) - u[i_0 + 1, i_1 - 1]/(2*\Delta x*\Delta y) - u[i_0 - 1, i_1 + 1]/(2*\Delta x*\Delta y) + u[i_0 - 1, i_1 - 1]/(2*\Delta x*\Delta y) + u[i_0 + 1, i_1]/\Delta x**2 + u[i_0 - 1, i_1]/\Delta x**2 + (-2*\Delta x**2 - 2*\Delta y**2)*u[i_0, i_1]/(\Delta x**2*\Delta y**2)

If you access to the symbols, that's what the second return value is for:

In [13]:
symbols

{'indices': [i_0, i_1], 'function': u, 'spacings': [\Delta x, \Delta y]}

In [15]:
i0, i1 = symbols['indices']
u = symbols['function']

from findiff.symbolics import Equation

eq = Equation(expr, 0)
eq

Eq(u[i_0, i_1 + 1]/\Delta y**2 + u[i_0, i_1 - 1]/\Delta y**2 + u[i_0 + 1, i_1 + 1]/(2*\Delta x*\Delta y) - u[i_0 + 1, i_1 - 1]/(2*\Delta x*\Delta y) - u[i_0 - 1, i_1 + 1]/(2*\Delta x*\Delta y) + u[i_0 - 1, i_1 - 1]/(2*\Delta x*\Delta y) + u[i_0 + 1, i_1]/\Delta x**2 + u[i_0 - 1, i_1]/\Delta x**2 + (-2*\Delta x**2 - 2*\Delta y**2)*u[i_0, i_1]/(\Delta x**2*\Delta y**2), 0)

In [16]:
eq.solve(u[i0, i1])

Eq(u[i_0, i_1], (2*\Delta x**2*u[i_0, i_1 + 1] + 2*\Delta x**2*u[i_0, i_1 - 1] + \Delta x*\Delta y*u[i_0 + 1, i_1 + 1] - \Delta x*\Delta y*u[i_0 + 1, i_1 - 1] - \Delta x*\Delta y*u[i_0 - 1, i_1 + 1] + \Delta x*\Delta y*u[i_0 - 1, i_1 - 1] + 2*\Delta y**2*u[i_0 + 1, i_1] + 2*\Delta y**2*u[i_0 - 1, i_1])/(4*(\Delta x**2 + \Delta y**2)))